### Charging Profile

In [99]:
from scipy.optimize import fsolve
import numpy as np


def cv_pwr(t: float, pm: float, k: float) -> float:
    return pm * np.exp(-k * t)


def cv_eng(t2, t1, pm, k) -> float:
    return (-1 / k) * (cv_pwr(t2, pm, k) - cv_pwr(t1, pm, k))


def cv_pwr_avg(t2, t1, pm, k) -> float:
    return cv_eng(t2, t1, pm, k) / (t2 - t1)


def charge_profile(
    soc: float,
    d_c: float,
    cap: float = 70,
    ev_mpi: float = 7.36,
    k: float = 0.1,
) -> list:
    """
    Calculate the charging profile for a given EV.

    Parameters
    ----------
    soc : float
        State of charge of the battery in %.
    d_c : float
        Desired charge of the battery in %.
    cap : float
        Capacity of the battery in kWh.
    ev_mpi : float
        Maximum power input of the EV in kW.
    k : float
        Charging curve constant.
        0.01-0.03 charge aggressively,
        0.05-0.1  prioritizing battery health and longevity
    cv : float
        Capacity at which the charging curve flattens in %.
    cc : float
        Capacity at which the charging curve starts in %.

    Returns
    -------
    list
        List of the charging profile in the format [time, power, charge].
    """
    # --------------------------------------------------------------------------
    # Calculate the charging profile
    # --------------------------------------------------------------------------
    # Calculate the charge required to reach the desired capacity
    cv = 0.8 * cap
    dc = d_c * cap

    # current charge
    c0 = soc * cap
    p0 = ev_mpi
    t0 = c0 / p0

    # first part of charge < 80% of cap
    c1 = max(min(dc, cv) - min(c0, cv), 0)  # charge [kWh] required
    p1 = 0 if c1 == 0 else ev_mpi  #          power [kW] required
    t1 = 0 if c1 == 0 else c1 / p1   #        time [h] required

    # second part of charge > 80% of cap
    c2 = max(max(dc, cv) - max(c0, cv), 0)  # charge [kWh] required
    p2 = 0  #                                 initial power [kW] required
    t2 = 0  #                                 initial time [h] required

    # Define the function for the given equation with specific ta, pm and k
    def zero_for_E(t2, t1, pm, k, E) -> float:
        return cv_eng(t2, t1, pm, k) - E

    # dtermime time and power for 2nd part of charge CV
    if c2 > 0:
        # Solve the equation numerically
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html
        solution = fsolve(func=zero_for_E, x0=t1, args=(0, ev_mpi, k, c2), xtol=1e-3)

        t2 = solution[0]  # time [h] required 2nd part CV
        p2 = c2 / t2  #     power [kW] required 2nd part CV

    # e2 = 0 if t2 == 0 else float(equation(t2))
    e2 = 0 if t2 == 0 else float(zero_for_E(t2=t2, t1=0, pm=ev_mpi, k=k, E=c2))

    return {
        "params": {"soc": soc, "d_c": d_c, "cap": cap, "ev_mpi": ev_mpi, "k": k},
        "phase0": {"c0": c0, "t": t0, "p": p0},
        "phase1": {"c1": c1, "t": t1, "p": p1},
        "phase2": {"c2": c2, "t": t2, "p": p2},
        "result": {"fc": c0 + c1 + c2 + e2, "dc": dc, "rm": dc - (c0 + c1 + c2 + e2)},
        "tslots": {"t1": t1, "t2": t1 + t2, "t3": t1 + t2},
    }

In [112]:
ev_mpi = 7.36
soc = 0.47
d_c = 1
cap = 70
k = 0.075

# Calculate the charging profile
cp = charge_profile(soc, d_c, cap, ev_mpi, k)


def myprint(cp):
    print(
        round(cp["params"]["soc"], 2),
        round(cp["params"]["d_c"], 2),
        cp["phase1"],
        cp["phase2"],
    )  # , cp["tslots"]["t3"])


print("\nFunction results\n")
# print(cp)
# myprint(cp)

for soc in np.arange(0.1, 1.1, 0.1):
    for d_c in np.arange(0.1, 1.1, 0.1):
        cp = charge_profile(soc, d_c, cap, ev_mpi, k)
        myprint(cp)


Function results

0.1 0.1 {'c1': 0.0, 't': 0, 'p': 0} {'c2': 0.0, 't': 0, 'p': 0}
0.1 0.2 {'c1': 7.0, 't': 0.9510869565217391, 'p': 7.36} {'c2': 0.0, 't': 0, 'p': 0}
0.1 0.3 {'c1': 14.000000000000004, 't': 1.9021739130434787, 'p': 7.36} {'c2': 0.0, 't': 0, 'p': 0}
0.1 0.4 {'c1': 21.0, 't': 2.8532608695652173, 'p': 7.36} {'c2': 0.0, 't': 0, 'p': 0}
0.1 0.5 {'c1': 28.0, 't': 3.8043478260869565, 'p': 7.36} {'c2': 0.0, 't': 0, 'p': 0}
0.1 0.6 {'c1': 35.0, 't': 4.755434782608695, 'p': 7.36} {'c2': 0.0, 't': 0, 'p': 0}
0.1 0.7 {'c1': 42.00000000000001, 't': 5.7065217391304355, 'p': 7.36} {'c2': 0.0, 't': 0, 'p': 0}
0.1 0.8 {'c1': 49.0, 't': 6.657608695652174, 'p': 7.36} {'c2': 0.0, 't': 0, 'p': 0}
0.1 0.9 {'c1': 49.0, 't': 6.657608695652174, 'p': 7.36} {'c2': 7.0, 't': 0.9867129483350375, 'p': 7.094261823371913}
0.1 1.0 {'c1': 49.0, 't': 6.657608695652174, 'p': 7.36} {'c2': 14.0, 't': 2.0523234224018143, 'p': 6.821536921123248}
0.2 0.1 {'c1': 0, 't': 0, 'p': 0} {'c2': 0.0, 't': 0, 'p': 0}
0

$$
\int_{a}^{b} P_{\text{max}} \times e^{-k \times (t - t_0)} \, dt = 
-\frac{P_{\text{max}}}{k} \left[ e^{-k \times (b - t_0)} - e^{-k \times (a - t_0)} \right]

$$
